In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})
df

,A,B,C,D
0,foo,one,-0.874009,-0.067027
1,bar,one,-0.648909,-0.053985
2,foo,two,-0.096479,0.024463
3,bar,three,-0.698216,1.187525
4,foo,two,0.918855,-0.081976
5,bar,two,1.734519,0.841634
6,foo,one,1.572423,0.423072
7,foo,three,0.904348,0.486459


In [2]:
g = df.groupby(['B', 'A'])
g

In [3]:
g.sum()
g.aggregate(np.sum) #equivalent

C         D
B     A                      
one   bar -0.648909 -0.053985
      foo  0.698413  0.356045
three bar -0.698216  1.187525
      foo  0.904348  0.486459
two   bar  1.734519  0.841634
      foo  0.822376 -0.057514

In [4]:
df.groupby(['A', 'B'], as_index=False).sum() # group, but don't make A and B indices
df.groupby(['A', 'B']).sum().reset_index() #equivalent

,A,B,C,D
0,bar,one,-0.648909,-0.053985
1,bar,three,-0.698216,1.187525
2,bar,two,1.734519,0.841634
3,foo,one,0.698413,0.356045
4,foo,three,0.904348,0.486459
5,foo,two,0.822376,-0.057514


In [5]:
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'

g = df.groupby(get_letter_type, axis=1)
g.get_group('consonant')
g.sum()


,consonant,vowel
0,-0.941037,foo
1,-0.702894,bar
2,-0.072016,foo
3,0.489310,bar
4,0.836879,foo
5,2.576153,bar
6,1.995495,foo
7,1.390807,foo


In [6]:
g.groups

{'consonant': Index([u'B', u'C', u'D'], dtype='object'),
 'vowel': Index([u'A'], dtype='object')}

In [7]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])

s = pd.Series(np.random.randn(8), index=index)

s

first  second
bar    one       0.515756
       two       0.456570
baz    one       0.424403
       two       0.204368
foo    one       0.361091
       two       0.406260
qux    one       0.251237
       two       0.630529
dtype: float64

In [8]:
s.groupby(level=0).sum()
s.groupby(level='second').sum() # same as level=1
s.sum(level='second') # same as line above

second
one    1.552488
two    1.697727
dtype: float64

In [9]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])

df = pd.DataFrame({'A': [1, 1, 1, 1, 2, 2, 3, 3],
                   'B': np.arange(8),
                   'C': np.arange(3, 11)},
                  index=index)
df

A  B   C
first second          
bar   one     1  0   3
      two     1  1   4
baz   one     1  2   5
      two     1  3   6
foo   one     2  4   7
      two     2  5   8
qux   one     3  6   9
      two     3  7  10

A DataFrame may be grouped by a combination of columns and index levels by specifying the column names as strings and the index levels as pd.Grouper objects.

In [10]:
# group by the 2nd index level and the 'A' column:
df.groupby([pd.Grouper(level=1), 'A']).sum()
# or, equivalently, feed in directly as keys to groupby()
df.groupby(['second', 'A']).sum()

B   C
second A       
one    1  2   8
       2  4   7
       3  6   9
two    1  4  10
       2  5   8
       3  7  10

In [11]:
df

A  B   C
first second          
bar   one     1  0   3
      two     1  1   4
baz   one     1  2   5
      two     1  3   6
foo   one     2  4   7
      two     2  5   8
qux   one     3  6   9
      two     3  7  10

In [41]:
grouped = df.groupby('A')
for name, group in grouped:
    print('name = {}'.format(name))
    print(group)
grouped.get_group(1)

name = 1
              A  B  C
first second         
bar   one     1  0  3
      two     1  1  4
baz   one     1  2  5
      two     1  3  6
name = 2
              A  B  C
first second         
foo   one     2  4  7
      two     2  5  8
name = 3
              A  B   C
first second          
qux   one     3  6   9
      two     3  7  10


A  B  C
first second         
bar   one     1  0  3
      two     1  1  4
baz   one     1  2  5
      two     1  3  6

In [54]:
df_re = pd.DataFrame({'A': [1] * 5 + [5] * 5,
                      'B': np.arange(10)})
print(df_re)
#whereas aggregate lowers the dims, transform functions dont

#aggregate
df_re.groupby('A').sum()

   A  B
0  1  0
1  1  1
2  1  2
3  1  3
4  1  4
5  5  5
6  5  6
7  5  7
8  5  8
9  5  9


,B
A,
1,10
5,35


In [56]:
#transform
df_re.groupby('A').expanding().sum()

A     B
A              
1 0   1.0   0.0
  1   2.0   1.0
  2   3.0   3.0
  3   4.0   6.0
  4   5.0  10.0
5 5   5.0   5.0
  6  10.0  11.0
  7  15.0  18.0
  8  20.0  26.0
  9  25.0  35.0

In [57]:
# filter functions return a subset of the original object
dff = pd.DataFrame({'A': np.arange(8), 'B': list('aabbbbcc')})
print(dff)

dff.groupby('B').filter(lambda x: len(x) > 2) #only pass groups with more than 2 rows.

   A  B
0  0  a
1  1  a
2  2  b
3  3  b
4  4  b
5  5  b
6  6  c
7  7  c


,A,B
2,2,b
3,3,b
4,4,b
5,5,b


In [58]:
dff.groupby('B').filter(lambda x: len(x) > 2, dropna=False) # fill w NaN instead of dropping..

,A,B
0,NaN,NaN
1,NaN,NaN
2,2.0,b
3,3.0,b
4,4.0,b
5,5.0,b
6,NaN,NaN
7,NaN,NaN


In [59]:
#apply can apply any function to each group
dff.groupby('B').apply(lambda x: x.describe())

A
B                
a count  2.000000
  mean   0.500000
  std    0.707107
  min    0.000000
  25%    0.250000
  50%    0.500000
  75%    0.750000
  max    1.000000
b count  4.000000
  mean   3.500000
  std    1.290994
  min    2.000000
  25%    2.750000
  50%    3.500000
  75%    4.250000
  max    5.000000
c count  2.000000
  mean   6.500000
  std    0.707107
  min    6.000000
  25%    6.250000
  50%    6.500000
  75%    6.750000
  max    7.000000

In [61]:
# variables of 'Categorical' class can be used as groupby keys
data = pd.Series(np.random.randn(100))

factor = pd.qcut(data, [0, .25, .5, .75, 1.])

data.groupby(factor).mean()

(-2.53, -0.781]     -1.264460
(-0.781, -0.0193]   -0.433101
(-0.0193, 0.53]      0.266350
(0.53, 2.139]        1.081322
dtype: float64